# Simulation Analysis Notebook

## Table of Contents
1. [Prologue: Setup and Imports](#prologue)
2. [Simulation Data Setup](#data_setup)
3. [Simulation Class Initialization](#simulation_class)
4. [Source Profiles and Input Power](#sources)
5. [Time Frame and Diagnostics](#time_frames)
6. [2D Cut Plot](#2d_cut)
7. [1D Profile Relaxation](#1d_profile)
8. [Space-Time Diagrams](#space_time)
9. [Time averaged 1D profile](#timeavg_1dprof)
10. [Volume Integral Quantities](#volume_integrals)
11. [2D Cut Movie](#2d_movie)


## 1. Prologue: Setup and Imports <a id="prologue"></a>
This section imports necessary libraries and custom routines required for the analysis.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys

# Configure plotting
plt.rcParams["figure.figsize"] = (6,4)

# Custom libraries and routines
home_dir = os.path.expanduser("~")
sys.path.append(home_dir+'/personal_gkyl_scripts/python_utilities')
from classes import Simulation, Species
from utils import *

## 2. Simulation Data Setup <a id="data_setup"></a>
This section sets up the path to the simulation data and its file prefix.

In [2]:
simdir = 'sim_data_dir_example/2x2v_example'
fileprefix = 'gk_bgk_im_asdex_selfOnly_2x2v_p1'

## 3. Simulation Class Initialization <a id="simulation_class"></a>
Initialize the `Simulation` class (see /python_utilities/classes/simulation.py) and set physical and geometric parameters.

The simulation class is made to contain every parameter and information about the simulation we want to analyze. It is meant to store geometry, metric, where the data are located and how they were generated. It will be used by all plot and post processing routines.

For now we pass many simulation parameters manually as not all simulation parameters are stored in the output file metadata. This will be adapted later to read directly from the lua script that ran the simulation.

In [3]:
simulation = Simulation()
simulation.set_phys_param(
    B_axis = 1.4,           # Magnetic field at magnetic axis [T]
    eps0 = 8.854e-12,       # Vacuum permittivity [F/m]
    eV = 1.602e-19,         # Elementary charge [C]
    mp = 1.673e-27,         # Proton mass [kg]
    me = 9.109e-31,         # Electron mass [kg]
)
simulation.set_geom_param(
    R_axis      = 0.8727315068,         # Magnetic axis major radius
    Z_axis      = 0.1414361745,         # Magnetic axis height
    R_LCFSmid   = 1.0968432365089495,   # Major radius of LCFS at the midplane
    a_shift     = 0.25,                 # Parameter in Shafranov shift
    kappa       = 1.5,                 # Elongation factor
    delta       = 0.3,                 # Triangularity factor
    q0          = None,                 # Safety factor (unused)
    x_LCFS      = 0.04,                 # position of the LCFS (= core domain width)
    x_out       = 0.08                  # SOL domain width
)
# Define the species
ion = Species(name='ion',
              m=2.01410177811*simulation.phys_param.mp, # Ion mass
              q=simulation.phys_param.eV,               # Ion charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
elc = Species(name='elc',
              m=simulation.phys_param.me, 
              q=-simulation.phys_param.eV, # Electron charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
# Add them to the simulation (we need to know this before setting up the data parameters)
simulation.add_species(ion)
simulation.add_species(elc)
# This call will set up the data structure of the simulation and set up a large dictionary 
# conaining the receipes of many post processing quantities, see simulation.data_param.info()
simulation.set_data_param( simdir = simdir, fileprefix = fileprefix, species = simulation.species)
# This is the first call that will load data. 
# If the paths are not set up correctly, the script will fail here.
simulation.geom_param.load_metric(simulation.data_param.fileprefix)

## 4. Source Profiles and Input Power <a id="sources"></a>
Define source profiles and calculate input power and particle flux.

The sources are passed to the previously defined parameters to the Simulation instance.

A small print will display the input of energy and particle by a volume integration of the source profiles using the the Jacobian loaded in the metric.

In [ ]:
n_srcOMP=2.4e23
x_srcOMP=0.0
Te_srcOMP=2 * simulation.species['elc'].T0
Ti_srcOMP=2 * simulation.species['ion'].T0
sigma_srcOMP=0.03 * simulation.geom_param.Lx
floor_src=1e-2
def custom_density_src_profile(x,y,z):
    return n_srcOMP * (np.exp(-((x - x_srcOMP) ** 2) / (2.0 * sigma_srcOMP ** 2)) + floor_src)
def custom_temp_src_profile_elc(x, y = None, z = None):
    mask = x < (x_srcOMP + 3 * sigma_srcOMP)
    fout = np.empty_like(x)
    fout[mask] = Te_srcOMP; fout[~mask] = Te_srcOMP * 3.0 / 8.0
    return fout  
def custom_temp_src_profile_ion( x, y = None, z = None):
    mask = x < (x_srcOMP + 3 * sigma_srcOMP)
    fout = np.empty_like(x)
    fout[mask] = Ti_srcOMP; fout[~mask] = Ti_srcOMP * 3.0 / 8.0
    return fout   
simulation.set_OMPsources(n_srcOMP=n_srcOMP,x_srcOMP=x_srcOMP,
                          Te_srcOMP=Te_srcOMP,Ti_srcOMP=Ti_srcOMP,
                          sigma_srcOMP=sigma_srcOMP,floor_src=floor_src,
                          density_src_profile=custom_density_src_profile,
                          temp_src_profile_elc=custom_temp_src_profile_elc,
                          temp_src_profile_ion=custom_temp_src_profile_ion)
print("Input power:    %g MW"%(simulation.get_input_power()/1e6))
print("Input particle: %g part/s"%(simulation.get_input_particle()))

## 5. Time Frame and normalization <a id="time_frames"></a>
Load available time frames and integrated moment diagnostics.

We can set up different units for the plots. These calls will adapt automatically all plotting routine, setting up the axes accordingly.

We also look for all available frames, i.e. we look for all number XX in "[fileprefix]-[fieldname]_XX.gkyl"

In [ ]:
simulation.normalize('t','mus') # time in micro-seconds
simulation.normalize('x','minor radius') # radial coordinate normalized by the minor radius (rho=r/a)
simulation.normalize('y','Larmor radius') # binormal in term of reference sound Larmor radius
simulation.normalize('z','pi') # parallel angle devided by pi
simulation.normalize('fluid velocities','thermal velocity') # fluid velocity moments are normalized by the thermal velocity
simulation.normalize('temperatures','eV') # temperatures in electron Volt
simulation.normalize('pressures','Pa') # pressures in Pascal
simulation.normalize('energies','MJ') # energies in mega Joules

fieldname = fileprefix+'-field' #e.g. we check the electrostatic field files.
sim_frames = find_available_frames(simulation,fieldname)
print("Time frames available from %g to %g"%(sim_frames[0],sim_frames[-1]))

## 6. 2D Cut Plot <a id="2d_cut"></a>
Generate a 2D cut plot at a specified plane and time frame.

In [ ]:
cut_dir = 'xz' # the plane we want to plot
cut_coord = 0.0 # the coordinate were the plan stands If normalized units are defined, 
# this number is in the normalize units. One can also specify a integer for array index 
# or 'avg' to get an average over the reduced dim.
time_frame = sim_frames[-1] # the time frame
fieldnames = ['ne', 'phi', 'Te', 'Ti'] # the fields to plot, see simulation.display_available_fields() (some may not work in 2x2v)
plot_2D_cut(simulation, cut_dir, cut_coord, time_frame, fieldnames)

## 7. 1D Profile Relaxation <a id="1d_profile"></a>
Plot the relaxation of 1D profiles over time.

In [ ]:
cdirection = 'x'
ccoords = [0.0, 0.0]
fieldnames=['ne', 'phi', 'Te', 'Ti']
plot_1D_time_evolution(simulation, cdirection, ccoords, fieldnames, sim_frames[1:10])

## 8. Space-Time Diagrams <a id="space_time"></a>
Generate space-time diagrams for specified fields. Very useful to see the dynamics without a movie

In [ ]:
cut_dir = 'x'
cut_coord = [0.0, 0.0]
fieldnames = ['ne', 'phi']
plot_1D_time_evolution(simulation, cut_dir, cut_coord, fieldnames, sim_frames[:], space_time=True)

## 9. Time averaged profiles <a id="timeavg_1dprof"></a>
Generate 1D field profiles by averaging over given time frames.

In [ ]:
figout = []
cdirection='x'; ccoords=['avg',0.0]
fieldnames = [['ne','ni'],['Te','Ti']]
time_frames = sim_frames[-50:]
plot_1D(simulation,cdirection,ccoords,fieldnames,time_frames, errorbar = False, figout = figout)

## 10. Volume Integral Quantities <a id="volume_integrals"></a>
Plot volume-integrated quantities such as energies over time.

In [ ]:
fieldnames = [['Wtote', 'Wtoti'],['ne','ni']] # This format allows to plot different fields on top of eachother
time_frames = sim_frames[::2] # recommended to avoid computing volume integral for each frames (can be long)
plot_volume_integral_vs_t(simulation, fieldnames, time_frames)

## 11. 2D Cut Movie <a id="2d_movie"></a>
Create a movie of 2D cuts over time.

In [ ]:
cut_dir = 'xz'
cut_coord = 0.0
time_frames = sim_frames[::4]
fieldnames = ['ne', 'phi', 'Te', 'Ti']
movieprefix = 'my_nice_movie'
make_2D_movie(simulation, cut_dir=cut_dir, cut_coord=cut_coord, time_frames=time_frames, fieldnames=fieldnames)